In [6]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from shapely.geometry import Point, Polygon

In [11]:
# input a polygon, the year of projection, the scenario and the confidence percentile
# If the polygon is totally on land, return the value of the closest avalable data point to the controid of the polygon
# If some part of the polygon is on the sea, return the average of the sea level rise of the points on the sea
def get_sea_level(polygon, year=2100, scenario='ssp370', percentile=50):
    # open the netCDF file
    file_path='data\\total_{}_medium_confidence_values.nc'.format(scenario, scenario)
    data = nc.Dataset(file_path,'r')
    # get the lons and lats
    lons = data.variables['lon'][:]
    lats = data.variables['lat'][:]
    # get the correct year index
    year_idx = (year - 2020)//10 
    # get the centorid of the polygon
    centroid = polygon.centroid
    lon, lat = centroid.x, centroid.y
    # first find the idxs of lons and lats that are close to the input lon
    lon_idxs = np.where(np.isclose(lons, lon, atol=20))[0]
    lat_idxs = np.where(np.isclose(lats, lat, atol=20))[0]
    # get the intersection of the two idxs
    idxs = np.intersect1d(lon_idxs, lat_idxs)

    if len(idxs) == 0:
        print("Please input a valid coordinate. The centroid is far away from the sea.")
        return None
    # check if the polygon is totally on land
    contains_sea=False
    total_sea_level = 0
    points_on_sea = 0
    for idx in idxs:
        # check if the point is in polygon
        if Point(lons[idx], lats[idx]).within(polygon):
            contains_sea = True
            total_sea_level += data.variables['sea_level_change'][percentile, year_idx, idx]
            points_on_sea += 1
    if contains_sea:
        print(f"The polygon contains {points_on_sea} points on the sea")
        return float(total_sea_level/points_on_sea)
    else:
        # then find the closest (lat,lon) pair to the input (lat,lon) among these idxs
        best_distance = np.inf
        closest_location_idx = None 
        km_per_degree_lat = 111
        km_per_degree_lon = 111*np.cos(np.radians(lat))
        for idx in idxs:
            curr_lon, curr_lat = lons[idx], lats[idx]
            squared_distance = (km_per_degree_lon*(curr_lon - lon))**2 + (km_per_degree_lat*(curr_lat - lat))**2
            if squared_distance < best_distance:
                best_distance = squared_distance
                closest_location_idx = idx
        # print the closest distance
        print(f"Closest available data point is at ({lons[closest_location_idx]}, {lats[closest_location_idx]})")
        print(f"The distance to the closest data point is {np.sqrt(best_distance)} km")
        
        
        # get the sea level data at that point
        sea_level = data.variables['sea_level_change'][percentile, year_idx, closest_location_idx]
        return float(sea_level)

In [12]:
# try polygon of costa rica
polygon = Polygon([(-85, 8), (-83, 8), (-83, 11), (-85, 11)])

get_sea_level(polygon, year=2100, scenario='ssp370', percentile=50)

The polygon contains 4 points on the sea


695.0